#### Import necessary packages for data processing and analysis and Load the posts DataFrame, Logistic Regression model, CountVectorizer model, StringIndexer model, and IDF model.


In [0]:

# Import packages
from pyspark.sql.functions import udf, col, concat, lit, monotonically_increasing_id, regexp_replace, lower, explode, split, collect_list, array_contains, instr
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, Normalizer, CountVectorizer, StringIndexer, StringIndexerModel, IDFModel
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.sql.types import ArrayType, StringType
from nltk.stem import PorterStemmer
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import IndexToString, StringIndexer
from pyspark.ml.feature import CountVectorizerModel

# Loading files and models
posts = spark.read.parquet('/mnt/mydatalakefilesystem/Capstone2/posts/*')

model = LogisticRegressionModel.load('/mnt/mydatalakefilesystem/Capstone2/Models/lr_model')
counterVec = CountVectorizerModel.load('/mnt/mydatalakefilesystem/Capstone2/Models/cv_model')
lables = StringIndexerModel.load('/mnt/mydatalakefilesystem/Capstone2/Models/stringindexer')
idf_model = IDFModel.load('/mnt/mydatalakefilesystem/Capstone2/Models/tfidf')

#### Perform data cleaning, tokenization, stop word removal, stemming, and TF-IDF transformation on the posts DataFrame.


In [0]:


# Data cleaning
df = posts[['id', 'Body', 'Title']]
df = df.withColumn("text", concat(col("Title"), lit(" "), col("Body")))

df = df.withColumn("text", regexp_replace("text", r"\W+", " ")) \
     .withColumn("text", regexp_replace("text", r"\b\w\b", "")) \
     .withColumn("text", regexp_replace("text", "_", " ")) \
     .withColumn("text", regexp_replace("text", r"\s+", " ")) \
     .withColumn("text", lower("text"))

df = df[df['text'].isNotNull()]
     

# Tokenization
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
tokenized = tokenizer.transform(df)

# Removing stop words
stopword_remover = StopWordsRemover(inputCol="tokens", outputCol="stop_words")
stopword = stopword_remover.transform(tokenized)

# Stemming
stemmer_func = udf(lambda words: [PorterStemmer().stem(word) for word in words], ArrayType(StringType()))
stemmed = stopword.withColumn("stemmed", stemmer_func(col("stop_words")))

# Removing additional stop words (highly frequent words based on EDA)
custom_stop_words = ['code', 'use', 'pre', 'get', 'want', 'like', 'thank', 'tri', 'work', 'way', 'need']

custom_remover = StopWordsRemover(inputCol="stemmed", outputCol="filtered", stopWords = custom_stop_words)
filtered = custom_remover.transform(stemmed)

# Count vectorizer
text_cv = counterVec.transform(filtered)

# Performing TF-IDF
final = idf_model.transform(text_cv)
     


##### Use the trained model to predict labels for new posts based on the final DataFrame.


In [0]:
# Predicting labels of new posts
final = final.cache()
predictions = model.transform(final)

##### Define a UDF to extract labels based on a probability threshold from the predictions DataFrame and apply it to create a new column with predicted labels.


In [0]:
@udf(ArrayType(StringType()))
def extract_labels(probabilities, threshold=0.2):
    labels = [int(i) for i, prob in enumerate(probabilities) if prob > threshold]
    return labels

output = predictions.withColumn("predicted_labels", extract_labels(predictions.probability))


##### Decode the predicted labels and join them back to the original DataFrame, also selecting specific columns for the final output.

In [0]:
indexer = lables
exploded = output.withColumn("value", explode("predicted_labels"))
numeric = exploded.withColumn("indexed_value", col("value").cast("integer"))

i2s = IndexToString(inputCol="indexed_value", outputCol="temp_decoder", labels=indexer.labels)
temp_decoded = i2s.transform(numeric)

temp = temp_decoded.withColumnRenamed("Body", "Body_two") \
    .groupBy("Body_two").agg(collect_list("temp_decoder").alias("labels_decoder"))

output = output.join(temp, output.Body == temp.Body_two, 'inner')

i2s = IndexToString(inputCol="prediction", outputCol="tag", labels=indexer.labels)
output = i2s.transform(output)

output = output[['id', 'prediction', 'tag', 'predicted_labels', 'labels_decoder']]

*Display the final output DataFrame*

In [0]:
display(output)

id,prediction,tag,predicted_labels,labels_decoder
10584799,13.0,.net,List(13),List(.net)
10591083,226.0,classnotfoundexception,"List(2, 226, 405, 785)","List(android, classnotfoundexception, annotations, manifest)"
10583147,123.0,pdf,"List(1, 123)","List(c#, pdf)"
10578702,307.0,nginx,"List(262, 307)","List(functional-programming, nginx)"
10588332,3.0,javascript,List(3),List(javascript)
9209700,2.0,android,"List(2, 312, 334)","List(android, opengl-es-2.0, shader)"
10585937,8.0,c++,List(8),List(c++)
9859019,2.0,android,"List(2, 23, 30, 119)","List(android, air-android, air, mobile)"
10576923,9.0,python,"List(4, 9)","List(php, python)"
10588884,11.0,vb.net,List(11),List(vb.net)


#### Saves the output DataFrame to a single CSV file with a specified path.

In [0]:
save = output[['id', 'prediction', 'tag']]


# define this function
def crt_sgl_file(result_path):
        # write the result to a folder container several files
        path = "/mnt/bd-project/Predictions/Temp_Parq/"
        save.write.option("delimiter", ",").option("header", "true").mode("overwrite").csv(path)

        # list the folder, find the csv file 
        filenames = dbutils.fs.ls(path)
        name = ''
        for filename in filenames:
            if filename.name.endswith('csv'):
                org_name = filename.name

        # copy the csv file to the path you want to save, in this example, we use  "/mnt/deBDProject/BI/ml_result.csv"
        dbutils.fs.cp(path + '/'+ org_name, result_path)

        # delete the folder
        dbutils.fs.rm(path, True)

        print('single file created')

#### Generate timestamp and save output to a single CSV file


In [0]:
from pyspark.sql.functions import col, concat_ws
from datetime import datetime

# Generate timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# run the function
result_path = "/mnt/mydatalakefilesystem/Capstone2/Predictions/predictions_{}.csv".format(timestamp)

crt_sgl_file(result_path)

single file created
